In [8]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sys; sys.path.append("/data/jerrylee/pjt/BIGFAM.v.2.0")
from src import frreg

In [9]:
cohort = "UKB" # UKB, GS

# Step 1. Load relatives phenotype data

In [10]:
# parameters
info_fn = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/relationship_information/relatives.formatted.info"
pheno_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/phenotype"
frreg_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/frreg"
output_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/po-sib"

In [11]:
# relative information format
df_pair = pd.read_csv(info_fn, sep='\t')
df_pair.head()


,DOR,rcode,relationship,volid,relid,volage,relage,volsex,relsex,Erx
0,1,SB,daughter-sister,1000094,3653174,65,64,F,F,0.75
1,1,NaN,NaN,1000220,1691267,64,64,F,F,NaN
2,1,NaN,NaN,1000286,1571411,53,70,F,F,NaN
3,1,NaN,NaN,1000295,1045127,60,41,F,F,NaN
4,1,NaN,NaN,1000476,3599303,50,51,F,M,NaN


In [12]:
pheno_fns = os.listdir(pheno_path)
len(pheno_fns)

106

In [13]:
# Do FR-reg
subgroups = ["PC", "SB"]

for fn in tqdm(pheno_fns):
    pheno = fn.split(".")[0]
    pheno_fn = f"{pheno_path}/{fn}"
    df_pheno = pd.read_csv(pheno_fn, sep="\t", names=["eid", "pheno"])
    df_pheno = frreg.remove_outliers(df_pheno, "pheno")
    
    for subgroup in subgroups:
        # DOR=1인 데이터 중 rcode="SB"만 남기고, 나머지 DOR은 유지
        df_pair_filtered = pd.concat([
            # DOR=1이면서 rcode="SB"인 데이터
            df_pair[(df_pair["DOR"] == 1) & (df_pair["rcode"] == subgroup)],
            # DOR이 1이 아닌 모든 데이터
            df_pair[df_pair["DOR"] != 1]
        ]).reset_index(drop=True)

        # merge pheno with relatives
        df_mrg = frreg.merge_pheno_info(df_pheno, df_pair_filtered)
        try:
            df_frreg, msgs = frreg.familial_relationship_regression_DOR(
                df_mrg.drop(columns=["rcode", "relationship", "Erx"]),
                n_bootstrap=100
            )
            
            df_frreg.to_csv(
                f"{output_path}/{pheno}.{subgroup}.frreg",
                sep='\t',
                index=False
            )
            
            print(
                f"SAVED in {output_path}/{pheno}.{subgroup}.frreg", 
                flush=True
            )
                
        except:
            continue
    break

100%|██████████| 100/100 [00:14<00:00,  6.90it/s]
/data/jerrylee/pjt/BIGFAM.v.2.0/src/frreg.py:174: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_frreg = (pd.concat([df_frreg, results])
100%|██████████| 100/100 [01:33<00:00,  1.07it/s]

SAVED in /data/jerrylee/pjt/BIGFAM.v.2.0/data/UKB/po-sib/White_blood_cell__leukocyte__count.PC.frreg



100%|██████████| 100/100 [00:18<00:00,  5.42it/s]
/data/jerrylee/pjt/BIGFAM.v.2.0/src/frreg.py:174: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_frreg = (pd.concat([df_frreg, results])
100%|██████████| 100/100 [01:33<00:00,  1.07it/s]

SAVED in /data/jerrylee/pjt/BIGFAM.v.2.0/data/UKB/po-sib/White_blood_cell__leukocyte__count.SB.frreg



  0%|          | 0/106 [05:05<?, ?it/s]
